In [1]:
EXPERIMENT = 'zzQuick/sis2'
# TIMESTAMP = '0731-1633'


In [2]:
import sys
import os
import json
sys.path.append(os.getcwd())

from environment import Environment

env = Environment()

import tensorflow as tf

import importlib
import pandas as pd

from experiment import Experiment
from dataset import Reader
import toolbox as tbx

from models import metrics


environment loaded


2023-08-01 13:34:26.703956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 13:34:26.765805: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
experiment_root = os.path.join(env.EXPERIMENT_ROOT, EXPERIMENT)

try:
    timestamp = TIMESTAMP
except:
    timestamp = max(os.listdir(os.path.join(experiment_root, 'ckpt')))

exp = Experiment(experiment_root, EXPERIMENT, timestamp, restore=True)


In [5]:
modules = []
for filename in os.listdir(os.path.join(exp.PROJECT_ROOT, 'models')):
    if filename.endswith('.py'):
        module_name = filename[:-3]
        modules.append(importlib.import_module("." + module_name, package='models'))

model = None
for module in modules:
    if module.__name__[-len(exp.MODEL_NAME):] == exp.MODEL_NAME:
        model = module.GAN(exp)
assert model is not None

generator:tf.keras.Model = model.generator
discriminator:tf.keras.Model = model.discriminator


In [6]:
dataset_reader = Reader(exp, 'evaluate.py')
train_dataset = dataset_reader.train_dataset
test_dataset = dataset_reader.test_dataset


datamodel.Reader called by evaluate.py
full train dataset: 610
selected random sample train: 50
full test dataset: 58
selected random sample val: 10


In [7]:
checkpoint = tf.train.Checkpoint(
    generator_optimizer=model.generator_optimizer,
    discriminator_optimizer=model.discriminator_optimizer,
    generator=generator,
    discriminator=discriminator,
    step=tf.Variable(0, dtype=tf.int64))

stepoffset = 0
latest_checkpoint = None

print('Trying to restore: ' + os.path.join(exp.output.CKPT))
latest_checkpoint = tf.train.latest_checkpoint(os.path.join(exp.output.CKPT))
checkpoint.restore(latest_checkpoint).expect_partial()
stepoffset = int(checkpoint.step)
print("Loaded checkpoint:", latest_checkpoint)
print("Continue at step:", stepoffset)


Trying to restore: /home/cb/sis2/experiments/zzQuick/sis2/ckpt/0731-1458
Loaded checkpoint: /home/cb/sis2/experiments/zzQuick/sis2/ckpt/0731-1458/ckpt-5
Continue at step: 499


In [12]:
importlib.reload(metrics)

def evaluate(test_ds:tf.data.Dataset) -> pd.DataFrame:

    metric_results = pd.DataFrame()

    for step, (target, input_image) in test_ds.enumerate():
        # print(input_image.shape)
        prediction = generator(input_image, training=False)

        for (single_target, single_prediction) in zip(target, prediction):

            metric = metrics.Metrics(single_target.numpy(), single_prediction.numpy())
            df = pd.DataFrame([metric.getall()])
            metric_results = pd.concat([metric_results, df], ignore_index=True)
            # metric_results.append(metric.getall())
    
    return metric_results


In [11]:
eval_results = evaluate(test_dataset)


In [16]:
eval_results.to_csv(os.path.join(exp.EXPERIMENT_ROOT, f'{EXPERIMENT.replace("/", "_")}_{timestamp}.csv'), index_label='index')
